In [ ]:
import os
import sys
import time
import warnings
from random import shuffle

import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from tensorflow.python.lib.io import file_io
from skimage.transform import resize
import numpy as np
import keras
from keras import backend as K
import cv2
from keras.models import Sequential
from scipy.stats import kurtosis, skew

warnings.filterwarnings("ignore")


In [ ]:
from keras.layers import Conv2DTranspose, Dropout, Activation, BatchNormalization, UpSampling2D, Concatenate
from keras.layers import Input, Conv1D, MaxPooling1D, concatenate, TimeDistributed, Flatten, LSTM, Dense, GlobalAveragePooling1D
from keras.models import Model

In [ ]:
# Initializing dataframes for training and validation.
x_train = pd.DataFrame(columns=[i for i in range(0, 12)])
y_train = pd.DataFrame(columns=[0])
x_val = pd.DataFrame(columns=[i for i in range(0, 12)])
y_val = pd.DataFrame(columns=[0])

train_sessions = ['Session01', 'Session05', 'Session06', 'Session07', 'Session12']
# sessions = ['Session01_tmp']
count = 0
count_res = 0
for session in train_sessions:
    print('Processing training {}'.format(session))
    df_data_arm = pd.read_csv('TrainingData/{}/armIMU.txt'.format(session),  delim_whitespace=True, header=None)
    df_data_wrist = pd.read_csv('TrainingData/{}/wristIMU.txt'.format(session), delim_whitespace=True, header=None)
    count += df_data_arm.shape[0]
    features = pd.concat([df_data_arm, df_data_wrist], axis=1, sort=False, ignore_index=True)
    count_res += features.shape[0]
    df_data_detection = pd.read_csv('TrainingData/{}/detection.txt'.format(session), header=None)
    x_train = x_train.append(other=features, ignore_index=True)
    y_train = y_train.append(other=df_data_detection, ignore_index=True)
print(count)
print(count_res)
x_train.to_csv('x_train.txt', header=False, index=False, sep='\t')
y_train.to_csv('y_train.txt', header=False, index=False, sep='\t')
print('Saved training features!')
    
val_sessions = ['Session13']
for session in val_sessions:
    print('Processing validation {}'.format(session))
    df_data_arm = pd.read_csv('TrainingData/{}/armIMU.txt'.format(session),  delim_whitespace=True, header=None)
    df_data_wrist = pd.read_csv('TrainingData/{}/wristIMU.txt'.format(session), delim_whitespace=True, header=None)
    features = pd.concat([df_data_arm, df_data_wrist], axis=1, sort=False, ignore_index=True)
    df_data_detection = pd.read_csv('TrainingData/{}/detection.txt'.format(session), header=None)
    x_val = x_val.append(other=features, ignore_index=True)
    y_val = y_val.append(other=df_data_detection, ignore_index=True)
    
x_val.to_csv('x_val.txt', header=False, index=False, sep='\t')
y_val.to_csv('y_val.txt', header=False, index=False, sep='\t')
print('Saved validation features!')

In [ ]:
x_train_df = np.loadtxt('x_train.txt', delimiter='\t')
y_train_df = np.loadtxt('y_train.txt', delimiter='\t')
x_val_df = np.loadtxt('x_val.txt', delimiter='\t')
y_val_df = np.loadtxt('y_val.txt', delimiter='\t')

In [ ]:
# x_zeroes = np.zeros((31, 12))
# y_zeroes = np.zeros((31,))
# x_train_df = np.concatenate((x_train_df, x_zeroes), axis=0)
# y_train_df = np.concatenate((y_train_df, y_zeroes), axis=0)
X_train_reshaped = np.reshape(x_train_df, (x_train_df.shape[0], 1, x_train_df.shape[1]))

# x_val_zeroes = np.zeros((10, 12))
# y_val_zeroes = np.zeros((10,))
# x_val_df = np.concatenate((x_val_df, x_val_zeroes), axis=0)
# y_val_df = np.concatenate((y_val_df, y_val_zeroes), axis=0)
X_val_reshaped = np.reshape(x_val_df, (x_val_df.shape[0], 1, x_val_df.shape[1]))


In [ ]:
print(X_val_reshaped.shape)
print(y_val_df.shape)

print(X_train_reshaped.shape)
print(y_train_df.shape)

## Build model

In [ ]:
data_dim = 12
timesteps = 1
batch_size = 1

In [ ]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, stateful=True,
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(32, return_sequences=True, stateful=True))
model.add(LSTM(32, stateful=True))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [ ]:
# define the checkpoint
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## Training

In [ ]:
history = model.fit(X_train_reshaped, y_train_df, batch_size=1, epochs=10, validation_data=(X_val_reshaped, y_val_df), callbacks=callbacks_list)


In [ ]:
score = model.evaluate(X_val_reshaped, y_val_df, batch_size=1)
print(list(zip(model.metrics_names,score)))